In [1]:
import requests

In [2]:
genome = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq").text
genome

'@ERR266411.1 HS18_09233:8:1307:10911:3848#168/1\nTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC\n+\nB@DFEFFFGEGGGHEHGHGHGGGGHIFGFIFHICFGHGHGJGHFGHGIHEHGGHJGFEFHGHEGGHHGHIFGFGDIFGGFGGGFHGGGHGGGAGIFGGCG\n@ERR266411.2 HS18_09233:8:2114:9377:50721#168/1\nAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC\n+\n=@@CEBF@BGBG6GF1E<04C3E7E.9G=H<H:HH;HBH;G69F7,7DG((EG8A8,-5-8,?,,,AE>C,B?,,,64$AC\'+\'=\'+4\'3*4+E322*\'(\n@ERR266411.3 HS18_09233:8:2116:11778:26937#168/1\nTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC\n+\n@@DFEEFFGFGGGFFHGHGGIGGGHIHGFIEHGHHHHFIGGEHFGHGFHGHEGHGGFGFDGHHGIHCGDFFDFDGEIGGFGHGGHGCFFGGGDGEFCFCD\n@ERR266411.4 HS18_09233:8:2204:14371:92031#168/1\nAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTCTG\n+\nA@DFEFFFGDGGGHFHHHGHGGFGHILGFGEHHHFHHFIGHGHFGHGGGEHGGHJGGHFGFGHHEHFGHI

In [3]:
def read_fastq(genome):
  sequences = []
  qualities = []
  for i, line in enumerate(genome.split("\n")):
    if i % 4 == 1:
      sequences.append(line)
    if i % 4 == 3:
      qualities.append(line)

  return sequences, qualities

In [4]:
sequences, qualities = read_fastq(genome)

In [5]:
sequences[:5], qualities[:5]

(['TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC',
  'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC',
  'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC',
  'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTCTG',
  'AGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATC'],
 ['B@DFEFFFGEGGGHEHGHGHGGGGHIFGFIFHICFGHGHGJGHFGHGIHEHGGHJGFEFHGHEGGHHGHIFGFGDIFGGFGGGFHGGGHGGGAGIFGGCG',
  "=@@CEBF@BGBG6GF1E<04C3E7E.9G=H<H:HH;HBH;G69F7,7DG((EG8A8,-5-8,?,,,AE>C,B?,,,64$AC'+'='+4'3*4+E322*'(",
  '@@DFEEFFGFGGGFFHGHGGIGGGHIHGFIEHGHHHHFIGGEHFGHGFHGHEGHGGFGFDGHHGIHCGDFFDFDGEIGGFGHGGHGCFFGGGDGEFCFCD',
  'A@DFEFFFGDGGGHFHHHGHGGFGHILGFGEHHHFHHFIGHGHFGHGGGEHGGHJGGHFGFGHHEHFGHICKFGJEIGGIFGGGHGGFHIEEGGEFGKGD',
  'B@DFEHFEGEGGGHGFGHGHFFGGGI@GDFHHFFHGHGGFHC

In [6]:
def naive(p, t):
  occurrences = []
  for i in range(len(t) - len(p) + 1):  # loop over alignments
    match = True
    for j in range(len(p)):  # loop over characters
      if t[i + j] != p[j]:  # compare characters
        match = False
        break

    if match:
      occurrences.append(i)  # all chars matched; record

  return occurrences

In [7]:
def reverse_complement(s):
  return s.translate(str.maketrans("ATCGN", "TAGCN"))[::-1]

In [8]:
def read_genome(s):
  genome = []
  for line in s.split("\n"):
    # ignore header line with genome information
    if not line[:1] == ">":
      genome.append(line)

  return "".join(genome)

In [9]:
def naive_with_rc(p, t):
  occurrences = naive(p, t)
  occurrences.extend(naive(reverse_complement(p), t))

  return list(set(occurrences))

In [10]:
genome2 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa").text

In [11]:
sequence2 = read_genome(genome2)
sequence2

'...'

In [12]:
naive("AGGTGGGAA", sequence2), naive(reverse_complement("AGGTGGGAA"), sequence2)

([28979], [])

In [13]:
naive_with_rc("AGGTGGGAA", sequence2), naive_with_rc(reverse_complement("AGGTGGGAA"), sequence2)

([28979], [28979])

In [14]:
sequence2.count("TTAA")

195

In [15]:
len(naive_with_rc("AGGT", sequence2))

306

In [16]:
len(naive_with_rc("TTAA", sequence2))

195

In [17]:
sorted(naive_with_rc("ACTAAGT", sequence2))[0]

26028

In [18]:
sorted(naive_with_rc("AGTCGA", sequence2))[0]

450

In [19]:
def naive_2mm(p, t):
  occurrences = []
  for i in range(len(t) - len(p) + 1):  # loop over alignments
    mismatches = 0
    for j in range(len(p)):  # loop over characters
      if t[i + j] != p[j]:  # compare characters
        mismatches += 1
        if mismatches > 2:
          break

    if mismatches <= 2:
      occurrences.append(i)  # all chars matched; record

  return occurrences

In [20]:
len(naive_2mm("TTCAAGCC", sequence2))

191

In [21]:
sorted(naive_2mm("AGGAGGTT", sequence2))[0]

49

In [22]:
genome3 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq").text
sequences3, qualities3 = read_fastq(genome3)

In [23]:
print(genome3[:955])

@ERR037900.1 509.8.8.8903.80024/1
TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTAAC
+
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHGFHHHFHFFHHHHHGHHFHEH@4#55554455HGFBF<@C>7EEF@FBEDDD<=C<E
@ERR037900.2 509.5.68.21343.17610/1
TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTNACCCTAAC
+
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCHHHHEHHBA#C>@54455C/7=CGHEGEB;C############
@ERR037900.3 509.5.41.1218.7494/1
TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTAAC
+
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHGHHHHHHHHHHHHHDHHHDEHHHHFGIHEHEGGGF4#45655366GIGEHAGBG################
@ERR037900.4 509.3.45.14457.64151/1
TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTACC
+
HHHHHHHHHHHHHHHHHHHHHHHHHIHHHHHHHHHHHHHHHHHHHHHHIHHHHHIHFHHHIHHHHD#ECA54655GGIBH?BD@+BCBF?5A=::>8?##


In [24]:
worst_cycle = 0
worst_cycle_quality = None

for c in range(len(sequences3[0])):
  cycle_quality = sum(ord(q[c]) - 33 for q in qualities3)
  # print(c, cycle_quality)
  if worst_cycle_quality is None or cycle_quality < worst_cycle_quality:
    worst_cycle = c
    worst_cycle_quality = cycle_quality

worst_cycle

66

In [25]:
from bm_preproc import BoyerMoore


def boyer_moore(p, p_bm, t):
  """Do Boyer-Moore matching. p=pattern, t=text,
    p_bm=BoyerMoore object for p"""
  i = 0
  occurrences = []
  while i < len(t) - len(p) + 1:
    shift = 1
    mismatched = False
    for j in range(len(p)-1, -1, -1):
      if p[j] != t[i+j]:
        skip_bc = p_bm.bad_character_rule(j, t[i+j])
        skip_gs = p_bm.good_suffix_rule(j)
        shift = max(shift, skip_bc, skip_gs)
        mismatched = True
        break
    if not mismatched:
      occurrences.append(i)
      skip_gs = p_bm.match_skip()
      shift = max(shift, skip_gs)
    i += shift

  return occurrences

In [26]:
genome4 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta").text

In [27]:
hch1 = read_genome(genome4)
len(hch1)

800000

In [28]:
def naive_debug(p, t):
  char_comps = 0
  alignments = 0

  occurrences = []
  for i in range(len(t) - len(p) + 1):  # loop over alignments
    match = True
    for j in range(len(p)):  # loop over characters
      char_comps += 1
      if t[i + j] != p[j]:  # compare characters
        match = False
        break

    alignments += 1
    if match:
      occurrences.append(i)  # all chars matched; record

  return occurrences, char_comps, alignments

In [29]:
def boyer_moore_debug(p, p_bm, t):
  """Do Boyer-Moore matching. p=pattern, t=text,
    p_bm=BoyerMoore object for p"""
  char_comps = 0
  alignments = 0

  i = 0
  occurrences = []
  while i < len(t) - len(p) + 1:
    shift = 1
    mismatched = False
    for j in range(len(p)-1, -1, -1):
      char_comps += 1
      if p[j] != t[i+j]:
        skip_bc = p_bm.bad_character_rule(j, t[i+j])
        skip_gs = p_bm.good_suffix_rule(j)
        shift = max(shift, skip_bc, skip_gs)
        mismatched = True
        break

    alignments += 1
    if not mismatched:
      occurrences.append(i)
      skip_gs = p_bm.match_skip()
      shift = max(shift, skip_gs)

    i += shift

  return occurrences, char_comps, alignments

In [30]:
naive_debug("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG", hch1)  # 799954, 984143, 799954

([56922], 984143, 799954)

In [31]:
naive_debug("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG", hch1)  # alignments: 799954, char_comps: 984143

([56922], 984143, 799954)

In [32]:
boyer_moore_debug("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG", BoyerMoore("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"), hch1)  # alignments: 127974, char_comps: 165191

([56922], 165191, 127974)

In [33]:
from kmer_index import Index

def approx_index_match(p, t, n, index):
  ...

In [34]:
len(approx_index_match("GGCGCGGTGGCTCACGCCTGTAAT", hch1, 2, Index(hch1, 8)))

TypeError: object of type 'NoneType' has no len()

In [ ]:
from subseq_index import SubseqIndex

def approx_subsense_index_match(p, t, n, index):
  ...

In [ ]:
len(approx_subsense_index_match("GGCGCGGTGGCTCACGCCTGTAAT", hch1, 2, SubseqIndex(hch1, 8, 3)))

TypeError: object of type 'NoneType' has no len()

In [ ]:
def overlap(a, b, min_length=3):
  """Return length of longest suffix of 'a' matching
    a prefix of 'b' that is at least 'min_length'
    characters long.  If no such overlap exists,
    return 0."""
  start = 0  # start all the way at the left
  while True:
    start = a.find(b[:min_length], start)  # look for b's suffx in a
    if start == -1:  # no more occurrences to right
      return 0
    # found occurrence; check for full suffix/prefix match
    if b.startswith(a[start:]):
      return len(a)-start
    start += 1  # move just past previous match

import itertools

def scs(ss):
  """Returns shortest common superstring of given
    strings, which must be the same length."""
  shortest_sup = None
  for ssperm in itertools.permutations(ss):
    sup = ssperm[0]  # superstring starts as first string
    for i in range(len(ss)-1):
      # overlap adjacent strings A and B in the permutation
      olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
      # add non-overlapping portion of B to superstring
      sup += ssperm[i+1][olen:]
    if shortest_sup is None or len(sup) < len(shortest_sup):
      shortest_sup = sup  # found shorter superstring

  return shortest_sup  # return shortest

In [ ]:
len(scs(["CCT", "CTT", "TGC", "TGG", "GAT", "ATT"]))

11

In [ ]:
genome5 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq").text
sequences5, qualities5 = read_fastq(genome5)

In [ ]:
genome6 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta").text
sequence6 = read_genome(genome6)

In [ ]:
def editDistance(x, y):
  # Create distance matrix
  D = []
  for i in range(len(x)+1):
    D.append([0]*(len(y)+1))

  # Initialize first row and column of matrix
  for i in range(len(x)+1):
    D[i][0] = i

  for i in range(len(y)+1):
    D[0][i] = 0

  # Fill in the rest of the matrix
  for i in range(1, len(x)+1):
    for j in range(1, len(y)+1):
      distHor = D[i][j-1] + 1
      distVer = D[i-1][j] + 1
      if x[i-1] == y[j-1]:
        distDiag = D[i-1][j-1]
      else:
        distDiag = D[i-1][j-1] + 1
      D[i][j] = min(distHor, distVer, distDiag)

  # Edit distance is the value in the bottom right corner of the matrix
  return min(D[-1])

In [ ]:
editDistance("GATTTACCAGATTGAG", sequence6)

2

In [ ]:
genome7 = requests.get("https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq").text
sequences7, qualities7 = read_fastq(genome7)